In [129]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [130]:
subject = '수학'
room_id = '6756635bccd21cdc9762967b'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [131]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [132]:
extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
splited_data = utils.split_sentences(extracted_data)
splited_data

[{'idx': 0, 'text': '어 안녕하세요'},
 {'idx': 1, 'text': '잘 들려요?'},
 {'idx': 2, 'text': '네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요'},
 {'idx': 3, 'text': '일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요'},
 {'idx': 4, 'text': '네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요.'},
 {'idx': 5, 'text': '빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요.'},
 {'idx': 6, 'text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
 {'idx': 7, 'text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요'},
 {'idx': 8, 'text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요'},
 {'idx': 9, 'text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요'},
 {'idx': 10, 'text': '점 i는 abc의 내심이다'},
 {'idx': 11, 'text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
 {'idx': 12, 'text': '그것도 맞는데 이게 정의가 있고 연준학생?'},
 {'idx': 13, 'text': '혹시 아까도 소리 들렸나요?'},
 {'idx': 14,
  'text': '아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요'},
 {'idx': 15,
  'text': '내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요'},
 {'idx': 16, 'text': '그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 17, 'text':

In [133]:
for item in splited_data:
    time_list = []
    text_list = item['text'].split()
    for text in text_list:
        for data in extracted_data:
            if text in data['text']:
                time_list.append(data['time_range'])
                # print('original:', data['text'])
                data['text'] = data['text'].replace(text, "", 1).strip()
                # print('text:', text)
                # print('remaining:', data['text'])
                break
            continue
    print(item['text'], set(time_list))

어 안녕하세요 {'1m 5.9s ~ 1m 7.7s'}
잘 들려요? {'1m 5.9s ~ 1m 7.7s'}
네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 {'1m 9.8s ~ 1m 34.8s'}
일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 {'1m 9.8s ~ 1m 34.8s'}
네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. {'1m 35.2s ~ 1m 38.8s', '1m 9.8s ~ 1m 34.8s'}
빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. {'1m 39.1s ~ 1m 57.9s'}
근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. {'1m 58.2s ~ 2m 4.3s'}
저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 {'2m 5.8s ~ 2m 34.7s'}
일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 {'2m 5.8s ~ 2m 34.7s'}
이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 {'2m 5.8s ~ 2m 34.7s'}
점 i는 abc의 내심이다 {'2m 5.8s ~ 2m 34.7s'}
그 혹시 내심의 정의가 뭔지 알고 있어요? {'2m 5.8s ~ 2m 34.7s'}
그것도 맞는데 이게 정의가 있고 연준학생? {'2m 38.3s ~ 2m 45.2s', '3m 19.1s ~ 3m 19.5s'}
혹시 아까도 소리 들렸나요? {'3m 20.2s ~ 3m 25.6s'}
아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 {'3m 25.6s ~ 3m 45.9s'}
내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 {'3m 48.3s ~ 4m 6.9s', '3m 25.6s ~ 3m 45.9s'}
그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 

In [5]:
chunks_with_overlap = utils.split_with_overlap(splited_data, chunk_size=30, overlap=5)

In [6]:
from prompt import question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_analyzer)
system_prompt = question_analyzer.QuestionAnalyzer(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [7]:
question_indices = utils.extract_question_indices(question_analysis_results)
question_context = utils.get_question_context(splited_data, question_indices)
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의

In [8]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [9]:
for (res,item) in zip(question_classifier_results, question_context):
    item['result'] = res
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.',
  'result': 'False'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠',
  'result': 'True'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거

In [10]:
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
final_feed

[{'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 17,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': '그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 무슨 말인지 이해가 좀 되나요? 그러면은 지금 이번 문제에서 연준학생이 해본거 보면 해외 한거를 보면은 지금 보면은 이 선본 ai를 이렇게 그어놓고 지금 필기 공유가 되나요 여기 보면은 여기 40도 있고 그 다음에 여기 25도 이렇게 있잖아요 이렇게 되는게 아니고 이게 이렇게 뇌접원 이렇게 그어지고 그러면은 지금 제가 갖고 있는 여기 선분 두 개가 같으니까 요거 두 개 길이 같단 말이에요 그러면은 여기를 X라고 하고 여기

In [11]:
from prompt import digging

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging)
system_prompt = digging.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [12]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    item['answer'] = res['answer']
    item['reason'] = res['reason']
final_feed

[{'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠',
  'result': '이해점검형 질문',
  'answer': '네, 내심은 삼각형의 내접원의 중심이고, 각 변에서의 거리가 같은 점이에요.',
  'reason': "질문은 학생이 내심의 정의를 알고 있는지 확인하는 것이며, '네' 또는 '아니오'로 대답할 수 있는 형태입니다. 따라서 이는 학생의 이해도를 확인하기 위한 이해점검형 질문입니다."},
 {'idx': 17,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': '그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 무슨 말인지 이해가 좀 되나요? 그러면은 지금 이번 문제에서 연준학생이 

In [13]:
with open(f"{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)